This script is for both local environment and CSC environment.
Select the corresponding paths to run either locally or in the CSC environment.

In [1]:
!which python

/projappl/project_2013104/pengyan1/venv/bin/python


In [2]:
import sys
import os

folder_path = os.path.abspath("/projappl/project_2013104/pengyan1/venv/lib/python3.10/site-packages")
if folder_path not in sys.path:
    sys.path.append(folder_path)

sys.path

['/PUHTI_TYKKY_FRQGCcR/miniconda/envs/env1/lib/python310.zip',
 '/PUHTI_TYKKY_FRQGCcR/miniconda/envs/env1/lib/python3.10',
 '/PUHTI_TYKKY_FRQGCcR/miniconda/envs/env1/lib/python3.10/lib-dynload',
 '',
 '/PUHTI_TYKKY_FRQGCcR/miniconda/envs/env1/lib/python3.10/site-packages',
 '/projappl/project_2013104/pengyan1/venv/lib/python3.10/site-packages']

In [ ]:
import torch
import torch.nn.functional as F
from torchmetrics import JaccardIndex
import torch_geometric.transforms as T
from torch_geometric.datasets import S3DIS
from torch_geometric.loader import DataLoader
from torch_geometric.typing import WITH_TORCH_CLUSTER
from pyg_pointnet2 import PyGPointNet2NoColor

if not WITH_TORCH_CLUSTER:
    quit("This example requires 'torch-cluster'")

In [4]:
print(torch.cuda.is_available())

True


In [11]:
test_area = 6  
path = "/scratch/project_2013104/datasets/s3dis/"

# take out colors
class SelectLast3Features:
    def __call__(self, data):
        # If data.x is defined, select only its last 3 features.
        if data.x is not None:
            data.x = data.x[:, -3:]
        return data

# transform and pre_transform
transform = T.Compose([
    T.RandomJitter(0.01),
    T.RandomRotate(15, axis=0),
    T.RandomRotate(15, axis=1),
    T.RandomRotate(15, axis=2),
    SelectLast3Features()
])
pre_transform = T.NormalizeScale()

In [12]:
train_dataset = S3DIS(root=path, test_area=test_area, train=True, force_reload=True, 
                      transform=transform,pre_transform=pre_transform)
test_dataset = S3DIS(root=path, test_area=test_area, train=False, force_reload=True, 
                     pre_transform=pre_transform)

Processing...
Done!
Processing...
Done!


In [13]:
batch_size=64
num_workers=10

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                          num_workers=num_workers, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                         num_workers=num_workers, pin_memory=True)

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = PyGPointNet2NoColor(train_dataset.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [18]:
def train():
    model.train()

    total_loss = correct_nodes = total_nodes = 0
    for i, data in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        correct_nodes += out.argmax(dim=1).eq(data.y).sum().item()
        total_nodes += data.num_nodes

        if (i + 1) % 10 == 0:
            print(f'[{i+1}/{len(train_loader)}] Loss: {total_loss / 10:.4f} '
                  f'Train Acc: {correct_nodes / total_nodes:.4f}')
            total_loss = correct_nodes = total_nodes = 0



In [19]:
@torch.no_grad()
def test(loader):
    model.eval()
    jaccard = JaccardIndex(num_classes=loader.dataset.num_classes, task="multiclass").to(device)
    
    for data in loader:
        data = data.to(device)
        outs = model(data)
        preds = outs.argmax(dim=-1)
        jaccard.update(preds, data.y)
    
    return jaccard.compute().item()


In [20]:
import time
for epoch in range(1, 46):
    # Track epoch start time
    start_time = time.perf_counter()
    train()
    iou = test(test_loader)
    # Calculate epoch duration
    epoch_time = time.perf_counter() - start_time
    
    # Print results with time
    print(f'Epoch: {epoch:02d}, Test IoU: {iou:.4f}, Time: {epoch_time:.2f}s')

[10/318] Loss: 2.1878 Train Acc: 0.4147
[20/318] Loss: 1.3738 Train Acc: 0.6184
[30/318] Loss: 1.2064 Train Acc: 0.6418
[40/318] Loss: 1.0874 Train Acc: 0.6618
[50/318] Loss: 0.9927 Train Acc: 0.6864
[60/318] Loss: 0.9937 Train Acc: 0.6917
[70/318] Loss: 0.9561 Train Acc: 0.7060
[80/318] Loss: 0.9073 Train Acc: 0.7074
[90/318] Loss: 0.8670 Train Acc: 0.7305
[100/318] Loss: 0.8643 Train Acc: 0.7183
[110/318] Loss: 0.8534 Train Acc: 0.7266
[120/318] Loss: 0.7967 Train Acc: 0.7473
[130/318] Loss: 0.8435 Train Acc: 0.7329
[140/318] Loss: 0.8247 Train Acc: 0.7277
[150/318] Loss: 0.7782 Train Acc: 0.7501
[160/318] Loss: 0.7844 Train Acc: 0.7468
[170/318] Loss: 0.8224 Train Acc: 0.7420
[180/318] Loss: 0.8475 Train Acc: 0.7255
[190/318] Loss: 0.8174 Train Acc: 0.7353
[200/318] Loss: 0.7537 Train Acc: 0.7610
[210/318] Loss: 0.8364 Train Acc: 0.7245
[220/318] Loss: 0.7712 Train Acc: 0.7483
[230/318] Loss: 0.7533 Train Acc: 0.7544
[240/318] Loss: 0.7492 Train Acc: 0.7581
[250/318] Loss: 0.7417 Tr

In [22]:

checkpoint_path = "/scratch/project_2013104/checkpoints/pointnet2_s3dis_pretransform_seg_x3_45_checkpoint.pth"

# Save model, optimizer state, and any other info needed
torch.save({
    'epoch': 45,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    #'loss': loss,
    #'test_accuracy': test_acc
}, checkpoint_path)

print("Checkpoint saved successfully!")

Checkpoint saved successfully!
